In [24]:
import string
import os
from util.api_key import generate_nonce, generate_signature
try:
    from urllib.parse import urlparse,urlunparse
except ImportError:
    from urlparse import urlparse,urlunparse
import time, hmac, hashlib


In [25]:
urlparse

<function urllib.parse.urlparse>

In [59]:
# Generates an API signature.
# A signature is HMAC_SHA256(secret, verb + path + nonce + data), hex encoded.
# Verb must be uppercased, url is relative, nonce must be an increasing 64-bit integer
# and the data, if present, must be JSON without whitespace between keys.
#
# For example, in psuedocode (and in real code below):
#
# verb=POST
# url=/api/v1/order
# nonce=1416993995705
# data={"symbol":"XBTZ14","quantity":1,"price":395.01}
# signature = HEX(HMAC_SHA256(secret, 'POST/api/v1/order1416993995705{"symbol":"XBTZ14","quantity":1,"price":395.01}'))
def generate_signature(secret, verb, url, nonce, data):
    """Generate a request signature compatible with BitMEX."""
    # Parse the url so we can remove the base and extract just the path.
    parsedURL = urlparse(url)
    print(parsedURL)
    path = parsedURL.path
    if parsedURL.query:
        path = path + '?' + parsedURL.query

    #print ("Computing HMAC: %s" % verb + path + str(nonce) + data)
    stringMessage = verb + path + str(nonce) + data
    message = stringMessage.encode('utf-8')
    signature = hmac.new(secret.encode('utf-8'), message, digestmod=hashlib.sha256).hexdigest()
    return signature


In [53]:
%load_ext autoreload
from bitmex_websocket import BitMEXWebsocket
import logging
from time import sleep

API_KEY = "wwvS30igJDo6Ksxa0h2EP1Eq"
API_SECRET = "-DOHRIUObpSQilqyr2y18YcTRi5NWFIV95du4i8rG4VveOBI"

# Basic use of websocket.
def run():
    logger = setup_logger()
  
    # Instantiating the WS will make it connect. Be sure to add your api_key/api_secret.
    ws = BitMEXWebsocket(endpoint="https://testnet.bitmex.com/api/v1", symbol="XBTUSD",
                         api_key=API_KEY, api_secret=API_SECRET)

    logger.info("Instrument data: %s" % ws.get_instrument())

    # Run forever
    while(ws.ws.sock.connected):
        logger.info("Ticker: %s" % ws.get_ticker())
        if ws.config['api_key']:
            logger.info("Funds: %s" % ws.funds())
        logger.info("Market Depth: %s" % ws.market_depth())
        logger.info("Recent Trades: %s\n\n" % ws.recent_trades())
        sleep(10)


def setup_logger():
    # Prints logger info to terminal
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)  # Change this to DEBUG if you want a lot more info
    ch = logging.StreamHandler()
    # create formatter
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    # add formatter to ch
    ch.setFormatter(formatter)
    logger.addHandler(ch)
    return logger




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
run()

AttributeError: module 'string' has no attribute 'join'